<a href="https://colab.research.google.com/github/OnurBasci/ARGENIT_Internship/blob/main/Chromosome_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

first deep learning project

imports


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision
import torchvision.transforms as transforms


drivea erisim

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


unzip the dataset

In [ ]:
!cp -r /content/gdrive/MyDrive/staj/chromosome_segmentation/clasify_dataset /content/datasets

In [ ]:
%cd /content/datasets/clasify_dataset
!unzip /content/datasets/cuted_image.zip

Class for custom dataset

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io

In [ ]:
class chromosomes(Dataset):
  def __init__(self, csv_file, root_dir, transform=None):
    self.annotations = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform
  def __len__(self):
    return len(self.annotations)
  
  def __getitem__(self, index):
    img_path = os.path.join(self.root_dir, str(pd.read_csv("/content/datasets/z_labels.csv").iloc[index, 0]).split(";")[0]) #the most stupid code ever
    image = io.imread(img_path)
    y_label = torch.tensor(int(pd.read_csv("/content/datasets/z_labels.csv").iloc[index, 0].split(";")[1]))

    if self.transform:
      image = self.transform(image)
    
    return (image, y_label)

In [ ]:
print(str(pd.read_csv("/content/datasets/z_labels.csv").iloc[71, 0]).split(";")[1])
#print(str(pd.read_csv("z_labels.csv").iloc[0]).split(";")[1])

23


custom datasetler icin

In [ ]:
batch_size = 64
dataset = chromosomes(csv_file = '/content/datasets/z_labels.csv', root_dir = "/content/datasets/clasify_dataset/cuted_image", transform = transforms.ToTensor())
train_set, test_set = torch.utils.data.random_split(dataset, [20000, 3000])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

Parameters

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = 2500 #50*50 size of images
num_classes = 24
learning_rate = 0.01
num_epochs = 24
in_channel = 3

googlenet model

In [ ]:
model = torchvision.models.googlenet(pretrained=True)
model.to(device)

Resnet model

In [ ]:
model = torchvision.models.resnet18()
model.to(device)

Mnist dataseti icin

In [ ]:
batch_size = 64
train_dataset = datasets.MNIST(
    root="dataset/",
    train=True,
    transform=transforms.ToTensor(),
    download = True
)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)
test_dataset = datasets.MNIST(
    root="dataset/",
    train = False,
    transform = transforms.ToTensor(),
    download=True
)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=True
)


define the neural network


In [ ]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [ ]:
for epoch in range(num_epochs):
  print(f"Epoch: {epoch}")
  for batch_idx, (data, targets) in enumerate(train_loader):
    print(batch_idx)
    #get data to cuda if possible
    data = data.to(device=device)
    targets = targets.to(device=device)

    #get to correct shape 28*28-> 784
    #data = data.reshape(data.shape[0], -1)

    #forward propagation
    scores = model(data)
    loss = criterion(scores, targets)

    #zero previous gradients
    optimizer.zero_grad()

    #back-propagation
    loss.backward()

    #Adam descent
    optimizer.step()

In [ ]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0

  #put he model in the evaluation mode
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x= x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

      print(
          f"Got {num_correct}/{num_samples} with accuracy"
          f"{float(num_correct) / float(num_samples) * 100:.2f}"
      )

      model.train()

In [ ]:
#check_accuracy(train_loader, model)
check_accuracy(test_loader, model)